In [2]:
from torch.utils.data import Dataset
import torch
from torchvision import transforms
from PIL import Image
import skimage.io
import numpy as np
import pandas as pd
import pretrainedmodels
from torch.utils.data import DataLoader
import time
import glob
import os
from multiprocessing import Pool

model_configs = {'inceptionv4':{
                   'input_size': 299,
                   'input_mean': [0.5],
                   'input_std' : [0.5]
                    },
                 'inceptionresnetv2':{
                   'input_size': 299,
                   'input_mean': [0.5],
                   'input_std' : [0.5]
                    },
                 'resnet152':{
                   'input_size': 224,
                   'input_mean': [0.485, 0.456, 0.406],
                   'input_std' : [0.229, 0.224, 0.225]
                    },
                  'vgg19_bn':{
                   'input_size': 224,
                   'input_mean': [0.485, 0.456, 0.406],
                   'input_std' : [0.229, 0.224, 0.225]
                    },
                 'se_resnext101_32x4d':{
                   'input_size': 224,
                   'input_mean': [0.485, 0.456, 0.406],
                   'input_std' : [0.229, 0.224, 0.225]
                    },
                 'polynet':{
                   'input_size': 331,
                   'input_mean': [0.485, 0.456, 0.406],
                   'input_std' : [0.229, 0.224, 0.225]
                    }
                }



class AtlasData(Dataset):
    def __init__(self, split, train = True, model = 'bninception'):
        self.split = split
        self.train = train
        self.train_str = 'train' if self.train else 'test'
        self.text_file = 'data/atlas_{}_split_{}.txt'.format(self.train_str, self.split)
        
        self.data = [[y for y in x.strip().split(' ')] for x in open(self.text_file, 'r').readlines()]
        self.imgs = [x[0] for x in self.data]
        self.labels = [[int(p) for p in x[1:]] for x in self.data]
        
        self.input_size = model_configs[model]['input_size']
        self.input_mean = model_configs[model]['input_mean']
        self.input_std = model_configs[model]['input_std']
        
        self.transforms = transforms.Compose([transforms.RandomRotation(180),
                                              transforms.Resize(self.input_size),
                                              transforms.ToTensor(),
                                              transforms.Normalize(self.input_mean, self.input_std),
                                            ])
        
    def dump_image(self, i):
        image = self.load_image_stack(self.imgs[i])
        image_name = "data/train/{}_{}.png".format(self.imgs[i], 'stacked')
        image.save(image_name)
        print("Saved", image_name)

    
    def load_image_stack(self, image_id):
        colors = ['red', 'green', 'blue', 'yellow']
        absolute_paths = ["data/train/{}_{}.png".format(image_id, color) for color in colors]
        
        images = [skimage.io.imread(path) for path in absolute_paths]
        
        image_red = images[0]
        image_green = images[1] + (images[3]/2).astype(np.uint8)
        image_blue = images[2] + (images[3]/2).astype(np.uint8)
        
        final_image = np.stack((image_red, image_green, image_blue), -1)
        to_display = Image.fromarray(final_image)
        return to_display
        
    def load_image(self, i):
        image_id = self.imgs[i]
        image_path = "data/train/{}_{}.png".format(image_id, 'stacked')
        image = Image.open(image_path)
        
        return image

    
    def __getitem__(self, i):
        image = self.load_image(i)
        image = self.transforms(image)
        
        label = self.labels[i]
        label_arr = np.zeros(28, dtype = np.float32)
        [np.put(label_arr, x, 1) for x in label]
        
        label_arr = torch.from_numpy(label_arr)
        
        return image, label_arr, label
        
    def __len__(self):
        return len(self.imgs)
    
class EvalAtlasData(Dataset):
    def __init__(self, model = 'bninception'):
        self.image_ids = sorted(set([x.split('_')[0] for x in os.listdir('data/test')]))
        
        self.input_size = model_configs[model]['input_size']
        self.input_mean = model_configs[model]['input_mean']
        self.input_std = model_configs[model]['input_std']
        
        self.transforms = transforms.Compose([transforms.Resize(self.input_size),
                                              transforms.ToTensor(),
                                              transforms.Normalize(self.input_mean, self.input_std),
                                            ])
        
        
    def load_image_stack(self, image_id):
        colors = ['red', 'green', 'blue', 'yellow']
        absolute_paths = ["data/test/{}_{}.png".format(image_id, color) for color in colors]
        
        images = [skimage.io.imread(path) for path in absolute_paths]
        
        image_red = images[0]
        image_green = images[1] + (images[3]/2).astype(np.uint8)
        image_blue = images[2] + (images[3]/2).astype(np.uint8)
        
        final_image = np.stack((image_red, image_green, image_blue), -1)
        to_display = Image.fromarray(final_image)
        return to_display
    
    def dump_image(self, i):
        image = self.load_image_stack(self.image_ids[i])
        image_name = "data/test/{}_{}.png".format(self.image_ids[i], 'stacked')
        image.save(image_name)
        print("Saved", image_name)
        
    def load_image(self, i):
        image_id = self.image_ids[i]
        image_path = "data/test/{}_{}.png".format(image_id, 'stacked')
        image = Image.open(image_path)
        
        return image, image_id
    
    def __len__(self):
        return len(self.image_ids)
        
    def __getitem__(self, i):
        image, image_id = self.load_image(i)
        image = self.transforms(image)
        
        return image_id, image



In [3]:
from torch.nn import Softmax
from sklearn.metrics import f1_score

softmax = Softmax()

def f1_micro(y_true, y_preds, thresh=0.5, eps=1e-20):
    preds_bin = y_preds > thresh # binary representation from probabilities (not relevant)
    truepos = preds_bin * y_true
    
    p = truepos.sum() / (preds_bin.sum() + eps) # take sums and calculate precision on scalars
    r = truepos.sum() / (y_true.sum() + eps) # take sums and calculate recall on scalars
    
    f1 = 2*p*r / (p+r+eps) # we calculate f1 on scalars
    return f1

def f1_macro(y_true, y_preds, thresh=0.5, eps=1e-20):
    preds_bin = y_preds > thresh # binary representation from probabilities (not relevant)
    truepos = preds_bin * y_true

    p = truepos.sum(axis=0) / (preds_bin.sum(axis=0) + eps) # sum along axis=0 (classes)
                                                            # and calculate precision array
    r = truepos.sum(axis=0) / (y_true.sum(axis=0) + eps)    # sum along axis=0 (classes) 
                                                            #  and calculate recall array

    f1 = 2*p*r / (p+r+eps) # we calculate f1 on arrays
    return np.mean(f1)

def generate_preds(model_name,threshold,suffix):
    model = pretrainedmodels.__dict__[model_name](num_classes = 1000, pretrained = 'imagenet')
    in_features = model.last_linear.in_features
    model.last_linear = torch.nn.Linear(in_features, 28)
    
    if model_name == 'polynet':
        model = torch.nn.DataParallel(model, device_ids = [0,1,2,3]).cuda()
        model.load_state_dict(torch.load('{}_0.pth.tar'.format(model_name))['state_dict'])
        model = model.eval()
    else:
        model.load_state_dict(torch.load('{}_0.pth.tar'.format(model_name))['state_dict'])
        model = model.eval()
        model.cuda()

    eval_data = EvalAtlasData(model = model_name)
    dataloader = DataLoader(eval_data, 1, False)
    
    preds = []
    for i, (image_id, images) in enumerate(dataloader):
        images = images.cuda()

        raw_predictions = (model(images))
        raw_predictions = softmax(raw_predictions)
        
        predictions = np.argwhere(raw_predictions.data[0] > threshold)
        try:
            num_predictions = len(predictions.data[0])
        except IndexError:
            num_predictions = 0

        print('-----------------------------------------------------')
        print(image_id[0])
        print('Raw Prediction', raw_predictions)
        if num_predictions == 0:
            print('No value passed the threshold')
            predictions = [np.argmax(raw_predictions.detach().cpu().numpy())]
            num_predictions = 1
            print("Prediction:", predictions)
            print("Number of predictions", num_predictions)
        else:
            predictions = predictions.data[0].tolist()
            print("Prediction:", predictions)
            print("Number of predictions", num_predictions)

        predicted = ' '.join('%d' % prediction for prediction in predictions)
        print(image_id[0])
        print(predicted)
        pred = dict(Id = image_id[0], Predicted = predicted)
        preds.append(pred)
        
    df = pd.DataFrame(preds)
    df.to_csv('{}_{}_{}.csv'.format(model_name, threshold, suffix), index = False)
    
    
def validate(model_name, split, threshold, with_softmax = True):
    
    softmax = Softmax()
    
    criterion = torch.nn.BCEWithLogitsLoss()
    val_dataset = AtlasData(split = 0, train = False, model = model_name)
    val_loader = DataLoader(val_dataset, batch_size = 1, shuffle = False)

    model = pretrainedmodels.__dict__[model_name](num_classes = 1000, pretrained = 'imagenet')
    in_features = model.last_linear.in_features
    model.last_linear = torch.nn.Linear(in_features, 28)
    
    if model_name == 'polynet':
        model = torch.nn.DataParallel(model, device_ids = [0,1,2,3]).cuda()
        model.load_state_dict(torch.load('{}_{}.pth.tar'.format(model_name, split))['state_dict'])
        model = model.eval()
    else:
        model.load_state_dict(torch.load('{}_{}.pth.tar'.format(model_name, split))['state_dict'])
        model = model.eval()
        model.cuda()
        
    losses = []
    y_pred = np.zeros(len(val_loader) * 28).reshape(len(val_loader), 28)
    y_true = np.zeros(len(val_loader) * 28).reshape(len(val_loader), 28)

    for i, (images, label_arrs, labels) in enumerate(val_loader):
        images = images.cuda()
        label_arrs_cuda = label_arrs.cuda()

        raw_predictions = model(images)
        outputs = raw_predictions.data
        
        if with_softmax:
            raw_predictions = softmax(raw_predictions)
        
        loss = criterion(outputs, label_arrs_cuda)
        losses.append(loss)
        
        predictions = np.arange(28)[raw_predictions.data[0] > 0.15]
        
        y_pred[i,:] = predictions
        y_true[i,:] = label_arrs
        
        if sum(predictions) == 0:
            prediction = np.argmax(raw_predictions.detach().cpu().numpy())
            predictions = np.zeros(28)
            np.put(predictions, prediction, 1)
        
        
        if i%10==0:
            print('Testing {}/{}: Loss {}'.format(i,len(val_loader),sum(losses)/len(losses)))
   
            
 
    score = f1_macro(y_true, y_pred)
    avg_loss = sum(losses)/len(losses)
    
    print("Avg Loss", avg_loss)
    print("Final Score", score)

    return avg_loss, score

# ResNext

In [73]:
model_name = 'se_resnext101_32x4d'
split = 0
threshold = 0
with_softmax = False

validate(model_name, split, threshold, with_softmax)

Testing 0/10449: Loss 0.32934480905532837
Testing 10/10449: Loss 0.042075265198946
Testing 20/10449: Loss 0.08197374641895294
Testing 30/10449: Loss 0.07570978254079819
Testing 40/10449: Loss 0.07516707479953766
Testing 50/10449: Loss 0.07419747114181519
Testing 60/10449: Loss 0.1040276437997818
Testing 70/10449: Loss 0.14067676663398743
Testing 80/10449: Loss 0.14957717061042786
Testing 90/10449: Loss 0.1465841829776764
Testing 100/10449: Loss 0.15092754364013672
Testing 110/10449: Loss 0.1582859307527542
Testing 120/10449: Loss 0.15943250060081482
Testing 130/10449: Loss 0.15812444686889648
Testing 140/10449: Loss 0.16512317955493927
Testing 150/10449: Loss 0.17811015248298645
Testing 160/10449: Loss 0.17931711673736572
Testing 170/10449: Loss 0.17541542649269104
Testing 180/10449: Loss 0.16876842081546783
Testing 190/10449: Loss 0.171004518866539
Testing 200/10449: Loss 0.17393556237220764
Testing 210/10449: Loss 0.1790536642074585
Testing 220/10449: Loss 0.18006554245948792
Testing

Testing 1860/10449: Loss 0.19377365708351135
Testing 1870/10449: Loss 0.19336840510368347
Testing 1880/10449: Loss 0.1927843987941742
Testing 1890/10449: Loss 0.19252292811870575
Testing 1900/10449: Loss 0.19196702539920807
Testing 1910/10449: Loss 0.19263304769992828
Testing 1920/10449: Loss 0.1920052468776703
Testing 1930/10449: Loss 0.19157153367996216
Testing 1940/10449: Loss 0.19152334332466125
Testing 1950/10449: Loss 0.19145098328590393
Testing 1960/10449: Loss 0.19223874807357788
Testing 1970/10449: Loss 0.19205276668071747
Testing 1980/10449: Loss 0.19166548550128937
Testing 1990/10449: Loss 0.19202058017253876
Testing 2000/10449: Loss 0.19162453711032867
Testing 2010/10449: Loss 0.19219978153705597
Testing 2020/10449: Loss 0.192414328455925
Testing 2030/10449: Loss 0.1923016607761383
Testing 2040/10449: Loss 0.19205276668071747
Testing 2050/10449: Loss 0.1922074407339096
Testing 2060/10449: Loss 0.19323411583900452
Testing 2070/10449: Loss 0.19351613521575928
Testing 2080/104

Testing 3700/10449: Loss 0.19423213601112366
Testing 3710/10449: Loss 0.19435006380081177
Testing 3720/10449: Loss 0.19401545822620392
Testing 3730/10449: Loss 0.1939268410205841
Testing 3740/10449: Loss 0.1936958283185959
Testing 3750/10449: Loss 0.19362668693065643
Testing 3760/10449: Loss 0.19399264454841614
Testing 3770/10449: Loss 0.19372573494911194
Testing 3780/10449: Loss 0.19388200342655182
Testing 3790/10449: Loss 0.19406835734844208
Testing 3800/10449: Loss 0.19423404335975647
Testing 3810/10449: Loss 0.19418051838874817
Testing 3820/10449: Loss 0.19397686421871185
Testing 3830/10449: Loss 0.19368837773799896
Testing 3840/10449: Loss 0.19359777867794037
Testing 3850/10449: Loss 0.19340699911117554
Testing 3860/10449: Loss 0.19328664243221283
Testing 3870/10449: Loss 0.19309844076633453
Testing 3880/10449: Loss 0.19316798448562622
Testing 3890/10449: Loss 0.19342593848705292
Testing 3900/10449: Loss 0.19313549995422363
Testing 3910/10449: Loss 0.19331297278404236
Testing 3920

Testing 5540/10449: Loss 0.19176462292671204
Testing 5550/10449: Loss 0.19178840517997742
Testing 5560/10449: Loss 0.19173263013362885
Testing 5570/10449: Loss 0.1918323040008545
Testing 5580/10449: Loss 0.1921463906764984
Testing 5590/10449: Loss 0.19199785590171814
Testing 5600/10449: Loss 0.1920902132987976
Testing 5610/10449: Loss 0.19199882447719574
Testing 5620/10449: Loss 0.1921026110649109
Testing 5630/10449: Loss 0.1919848769903183
Testing 5640/10449: Loss 0.1918347179889679
Testing 5650/10449: Loss 0.19174057245254517
Testing 5660/10449: Loss 0.19160450994968414
Testing 5670/10449: Loss 0.19170771539211273
Testing 5680/10449: Loss 0.19152724742889404
Testing 5690/10449: Loss 0.19171348214149475
Testing 5700/10449: Loss 0.1917038857936859
Testing 5710/10449: Loss 0.1917145550251007
Testing 5720/10449: Loss 0.19175943732261658
Testing 5730/10449: Loss 0.1916835457086563
Testing 5740/10449: Loss 0.19151650369167328
Testing 5750/10449: Loss 0.19156032800674438
Testing 5760/10449:

Testing 7380/10449: Loss 0.19108828902244568
Testing 7390/10449: Loss 0.19106745719909668
Testing 7400/10449: Loss 0.19104884564876556
Testing 7410/10449: Loss 0.19125258922576904
Testing 7420/10449: Loss 0.19119705259799957
Testing 7430/10449: Loss 0.19110877811908722
Testing 7440/10449: Loss 0.19115127623081207
Testing 7450/10449: Loss 0.1914130002260208
Testing 7460/10449: Loss 0.191473126411438
Testing 7470/10449: Loss 0.1915517896413803
Testing 7480/10449: Loss 0.19145089387893677
Testing 7490/10449: Loss 0.19137464463710785
Testing 7500/10449: Loss 0.19139046967029572
Testing 7510/10449: Loss 0.19126397371292114
Testing 7520/10449: Loss 0.1912204921245575
Testing 7530/10449: Loss 0.19103889167308807
Testing 7540/10449: Loss 0.19111645221710205
Testing 7550/10449: Loss 0.19127176702022552
Testing 7560/10449: Loss 0.19128422439098358
Testing 7570/10449: Loss 0.19142615795135498
Testing 7580/10449: Loss 0.19142085313796997
Testing 7590/10449: Loss 0.19151918590068817
Testing 7600/10

Testing 9220/10449: Loss 0.1919305920600891
Testing 9230/10449: Loss 0.19195324182510376
Testing 9240/10449: Loss 0.19209472835063934
Testing 9250/10449: Loss 0.19221706688404083
Testing 9260/10449: Loss 0.19233034551143646
Testing 9270/10449: Loss 0.19228601455688477
Testing 9280/10449: Loss 0.19231416285037994
Testing 9290/10449: Loss 0.19236469268798828
Testing 9300/10449: Loss 0.1923850029706955
Testing 9310/10449: Loss 0.19242340326309204
Testing 9320/10449: Loss 0.19238467514514923
Testing 9330/10449: Loss 0.19231100380420685
Testing 9340/10449: Loss 0.1922536939382553
Testing 9350/10449: Loss 0.19225656986236572
Testing 9360/10449: Loss 0.1922806352376938
Testing 9370/10449: Loss 0.19252344965934753
Testing 9380/10449: Loss 0.19246631860733032
Testing 9390/10449: Loss 0.19260552525520325
Testing 9400/10449: Loss 0.19253623485565186
Testing 9410/10449: Loss 0.19251415133476257
Testing 9420/10449: Loss 0.19235113263130188
Testing 9430/10449: Loss 0.19226615130901337
Testing 9440/1

(tensor(0.1937, device='cuda:0'), 0.5214877194895985)

In [74]:
split = 0
threshold = 0.1
with_softmax = False

validate(model_name, split, threshold, with_softmax)

Testing 0/10449: Loss 0.19498775899410248
Testing 10/10449: Loss 0.034829504787921906
Testing 20/10449: Loss 0.07398609071969986
Testing 30/10449: Loss 0.0796317309141159
Testing 40/10449: Loss 0.07471680641174316
Testing 50/10449: Loss 0.07552526146173477
Testing 60/10449: Loss 0.10733915120363235
Testing 70/10449: Loss 0.13349996507167816
Testing 80/10449: Loss 0.13795778155326843
Testing 90/10449: Loss 0.13349653780460358
Testing 100/10449: Loss 0.13487769663333893
Testing 110/10449: Loss 0.1442030966281891
Testing 120/10449: Loss 0.15049102902412415
Testing 130/10449: Loss 0.14882196485996246
Testing 140/10449: Loss 0.15181469917297363
Testing 150/10449: Loss 0.16244694590568542
Testing 160/10449: Loss 0.16638265550136566
Testing 170/10449: Loss 0.1624993532896042
Testing 180/10449: Loss 0.15684308111667633
Testing 190/10449: Loss 0.15965990722179413
Testing 200/10449: Loss 0.16305410861968994
Testing 210/10449: Loss 0.1688666045665741
Testing 220/10449: Loss 0.17051240801811218
Te

Testing 1860/10449: Loss 0.19534897804260254
Testing 1870/10449: Loss 0.19499549269676208
Testing 1880/10449: Loss 0.19465379416942596
Testing 1890/10449: Loss 0.19438625872135162
Testing 1900/10449: Loss 0.19385476410388947
Testing 1910/10449: Loss 0.19453950226306915
Testing 1920/10449: Loss 0.19402974843978882
Testing 1930/10449: Loss 0.19372710585594177
Testing 1940/10449: Loss 0.1935877799987793
Testing 1950/10449: Loss 0.19329632818698883
Testing 1960/10449: Loss 0.19474726915359497
Testing 1970/10449: Loss 0.19441695511341095
Testing 1980/10449: Loss 0.1940184384584427
Testing 1990/10449: Loss 0.19426195323467255
Testing 2000/10449: Loss 0.19387564063072205
Testing 2010/10449: Loss 0.19454115629196167
Testing 2020/10449: Loss 0.19499756395816803
Testing 2030/10449: Loss 0.1949131339788437
Testing 2040/10449: Loss 0.1946873515844345
Testing 2050/10449: Loss 0.19485148787498474
Testing 2060/10449: Loss 0.19549807906150818
Testing 2070/10449: Loss 0.19569993019104004
Testing 2080/1

Testing 3700/10449: Loss 0.19425997138023376
Testing 3710/10449: Loss 0.19436292350292206
Testing 3720/10449: Loss 0.1940215677022934
Testing 3730/10449: Loss 0.19390630722045898
Testing 3740/10449: Loss 0.19370678067207336
Testing 3750/10449: Loss 0.1935439109802246
Testing 3760/10449: Loss 0.19396673142910004
Testing 3770/10449: Loss 0.19363929331302643
Testing 3780/10449: Loss 0.1937313675880432
Testing 3790/10449: Loss 0.1938791126012802
Testing 3800/10449: Loss 0.19383947551250458
Testing 3810/10449: Loss 0.19367656111717224
Testing 3820/10449: Loss 0.19341424107551575
Testing 3830/10449: Loss 0.19306223094463348
Testing 3840/10449: Loss 0.19300678372383118
Testing 3850/10449: Loss 0.19276995956897736
Testing 3860/10449: Loss 0.19252105057239532
Testing 3870/10449: Loss 0.19226539134979248
Testing 3880/10449: Loss 0.19222265481948853
Testing 3890/10449: Loss 0.19227997958660126
Testing 3900/10449: Loss 0.19205792248249054
Testing 3910/10449: Loss 0.19225721061229706
Testing 3920/1

Testing 5540/10449: Loss 0.1902984082698822
Testing 5550/10449: Loss 0.19021402299404144
Testing 5560/10449: Loss 0.19014257192611694
Testing 5570/10449: Loss 0.19024613499641418
Testing 5580/10449: Loss 0.19065353274345398
Testing 5590/10449: Loss 0.19052499532699585
Testing 5600/10449: Loss 0.19052359461784363
Testing 5610/10449: Loss 0.19045792520046234
Testing 5620/10449: Loss 0.19049105048179626
Testing 5630/10449: Loss 0.1904548704624176
Testing 5640/10449: Loss 0.19032514095306396
Testing 5650/10449: Loss 0.19026273488998413
Testing 5660/10449: Loss 0.19013702869415283
Testing 5670/10449: Loss 0.1902248114347458
Testing 5680/10449: Loss 0.19004559516906738
Testing 5690/10449: Loss 0.19015493988990784
Testing 5700/10449: Loss 0.19012635946273804
Testing 5710/10449: Loss 0.1901737004518509
Testing 5720/10449: Loss 0.19025804102420807
Testing 5730/10449: Loss 0.19017617404460907
Testing 5740/10449: Loss 0.18997955322265625
Testing 5750/10449: Loss 0.19009935855865479
Testing 5760/1

Testing 7380/10449: Loss 0.18953390419483185
Testing 7390/10449: Loss 0.18945328891277313
Testing 7400/10449: Loss 0.1894475519657135
Testing 7410/10449: Loss 0.18961012363433838
Testing 7420/10449: Loss 0.1895606964826584
Testing 7430/10449: Loss 0.1894426941871643
Testing 7440/10449: Loss 0.18949969112873077
Testing 7450/10449: Loss 0.18982632458209991
Testing 7460/10449: Loss 0.18995746970176697
Testing 7470/10449: Loss 0.18997986614704132
Testing 7480/10449: Loss 0.18987318873405457
Testing 7490/10449: Loss 0.18982580304145813
Testing 7500/10449: Loss 0.18984834849834442
Testing 7510/10449: Loss 0.18976375460624695
Testing 7520/10449: Loss 0.18974542617797852
Testing 7530/10449: Loss 0.18954958021640778
Testing 7540/10449: Loss 0.18948069214820862
Testing 7550/10449: Loss 0.18952220678329468
Testing 7560/10449: Loss 0.18950460851192474
Testing 7570/10449: Loss 0.18964052200317383
Testing 7580/10449: Loss 0.18958979845046997
Testing 7590/10449: Loss 0.18980710208415985
Testing 7600/

Testing 9220/10449: Loss 0.19180099666118622
Testing 9230/10449: Loss 0.19189976155757904
Testing 9240/10449: Loss 0.19206783175468445
Testing 9250/10449: Loss 0.19223275780677795
Testing 9260/10449: Loss 0.1923566460609436
Testing 9270/10449: Loss 0.19230708479881287
Testing 9280/10449: Loss 0.19241583347320557
Testing 9290/10449: Loss 0.19247476756572723
Testing 9300/10449: Loss 0.1924469769001007
Testing 9310/10449: Loss 0.1924895942211151
Testing 9320/10449: Loss 0.1924760639667511
Testing 9330/10449: Loss 0.19235171377658844
Testing 9340/10449: Loss 0.19227053225040436
Testing 9350/10449: Loss 0.19229404628276825
Testing 9360/10449: Loss 0.19236235320568085
Testing 9370/10449: Loss 0.19256865978240967
Testing 9380/10449: Loss 0.19250524044036865
Testing 9390/10449: Loss 0.19262441992759705
Testing 9400/10449: Loss 0.1926039755344391
Testing 9410/10449: Loss 0.19258475303649902
Testing 9420/10449: Loss 0.19246046245098114
Testing 9430/10449: Loss 0.19237832725048065
Testing 9440/10

(tensor(0.1937, device='cuda:0'), 0.537531332669723)

In [75]:
split = 0
threshold = 0.15
with_softmax = False

validate(model_name, split, threshold, with_softmax)

Testing 0/10449: Loss 0.11123757809400558
Testing 10/10449: Loss 0.04445001482963562
Testing 20/10449: Loss 0.09918825328350067
Testing 30/10449: Loss 0.08988741040229797
Testing 40/10449: Loss 0.10176236182451248
Testing 50/10449: Loss 0.0856429934501648
Testing 60/10449: Loss 0.11920961737632751
Testing 70/10449: Loss 0.14766152203083038
Testing 80/10449: Loss 0.15860484540462494
Testing 90/10449: Loss 0.1526658684015274
Testing 100/10449: Loss 0.16758795082569122
Testing 110/10449: Loss 0.16480416059494019
Testing 120/10449: Loss 0.16601493954658508
Testing 130/10449: Loss 0.1676560938358307
Testing 140/10449: Loss 0.1707530915737152
Testing 150/10449: Loss 0.17903028428554535
Testing 160/10449: Loss 0.18452706933021545
Testing 170/10449: Loss 0.18067248165607452
Testing 180/10449: Loss 0.17364279925823212
Testing 190/10449: Loss 0.17670848965644836
Testing 200/10449: Loss 0.17846551537513733
Testing 210/10449: Loss 0.18202480673789978
Testing 220/10449: Loss 0.18627072870731354
Tes

Testing 1860/10449: Loss 0.19607789814472198
Testing 1870/10449: Loss 0.19580064713954926
Testing 1880/10449: Loss 0.19541427493095398
Testing 1890/10449: Loss 0.1958741843700409
Testing 1900/10449: Loss 0.19501718878746033
Testing 1910/10449: Loss 0.1955852061510086
Testing 1920/10449: Loss 0.1951572448015213
Testing 1930/10449: Loss 0.19466552138328552
Testing 1940/10449: Loss 0.19462478160858154
Testing 1950/10449: Loss 0.19464482367038727
Testing 1960/10449: Loss 0.19600747525691986
Testing 1970/10449: Loss 0.1961442530155182
Testing 1980/10449: Loss 0.19562354683876038
Testing 1990/10449: Loss 0.1962663233280182
Testing 2000/10449: Loss 0.196165531873703
Testing 2010/10449: Loss 0.19712184369564056
Testing 2020/10449: Loss 0.19755242764949799
Testing 2030/10449: Loss 0.19734624028205872
Testing 2040/10449: Loss 0.19690275192260742
Testing 2050/10449: Loss 0.19665133953094482
Testing 2060/10449: Loss 0.19719761610031128
Testing 2070/10449: Loss 0.19796337187290192
Testing 2080/1044

Testing 3700/10449: Loss 0.19555173814296722
Testing 3710/10449: Loss 0.19561299681663513
Testing 3720/10449: Loss 0.1952512115240097
Testing 3730/10449: Loss 0.1951911300420761
Testing 3740/10449: Loss 0.19489768147468567
Testing 3750/10449: Loss 0.1950507014989853
Testing 3760/10449: Loss 0.1957470327615738
Testing 3770/10449: Loss 0.19543983042240143
Testing 3780/10449: Loss 0.19552268087863922
Testing 3790/10449: Loss 0.19546957314014435
Testing 3800/10449: Loss 0.19575469195842743
Testing 3810/10449: Loss 0.195540651679039
Testing 3820/10449: Loss 0.19551481306552887
Testing 3830/10449: Loss 0.19535472989082336
Testing 3840/10449: Loss 0.19547946751117706
Testing 3850/10449: Loss 0.19532370567321777
Testing 3860/10449: Loss 0.19510142505168915
Testing 3870/10449: Loss 0.19506752490997314
Testing 3880/10449: Loss 0.19482219219207764
Testing 3890/10449: Loss 0.19489805400371552
Testing 3900/10449: Loss 0.19451352953910828
Testing 3910/10449: Loss 0.19461707770824432
Testing 3920/104

Testing 5540/10449: Loss 0.1963186413049698
Testing 5550/10449: Loss 0.19620466232299805
Testing 5560/10449: Loss 0.19622290134429932
Testing 5570/10449: Loss 0.19636324048042297
Testing 5580/10449: Loss 0.19661113619804382
Testing 5590/10449: Loss 0.1964598000049591
Testing 5600/10449: Loss 0.19637848436832428
Testing 5610/10449: Loss 0.19621829688549042
Testing 5620/10449: Loss 0.19622817635536194
Testing 5630/10449: Loss 0.19610066711902618
Testing 5640/10449: Loss 0.19602200388908386
Testing 5650/10449: Loss 0.1960529386997223
Testing 5660/10449: Loss 0.1959337294101715
Testing 5670/10449: Loss 0.19606056809425354
Testing 5680/10449: Loss 0.19600369036197662
Testing 5690/10449: Loss 0.19614458084106445
Testing 5700/10449: Loss 0.19605207443237305
Testing 5710/10449: Loss 0.1959872543811798
Testing 5720/10449: Loss 0.19599361717700958
Testing 5730/10449: Loss 0.19593070447444916
Testing 5740/10449: Loss 0.19574391841888428
Testing 5750/10449: Loss 0.19570392370224
Testing 5760/10449

Testing 7380/10449: Loss 0.1964474469423294
Testing 7390/10449: Loss 0.19639112055301666
Testing 7400/10449: Loss 0.19631333649158478
Testing 7410/10449: Loss 0.1965576410293579
Testing 7420/10449: Loss 0.19643422961235046
Testing 7430/10449: Loss 0.19640900194644928
Testing 7440/10449: Loss 0.19641196727752686
Testing 7450/10449: Loss 0.1965608298778534
Testing 7460/10449: Loss 0.19652225077152252
Testing 7470/10449: Loss 0.1965394765138626
Testing 7480/10449: Loss 0.19651752710342407
Testing 7490/10449: Loss 0.19648662209510803
Testing 7500/10449: Loss 0.19646118581295013
Testing 7510/10449: Loss 0.19637435674667358
Testing 7520/10449: Loss 0.19627626240253448
Testing 7530/10449: Loss 0.19608858227729797
Testing 7540/10449: Loss 0.19608555734157562
Testing 7550/10449: Loss 0.19603057205677032
Testing 7560/10449: Loss 0.19603899121284485
Testing 7570/10449: Loss 0.19617384672164917
Testing 7580/10449: Loss 0.19606615602970123
Testing 7590/10449: Loss 0.1962706297636032
Testing 7600/10

Testing 9220/10449: Loss 0.1980253905057907
Testing 9230/10449: Loss 0.19811773300170898
Testing 9240/10449: Loss 0.1982659101486206
Testing 9250/10449: Loss 0.1983877718448639
Testing 9260/10449: Loss 0.198383167386055
Testing 9270/10449: Loss 0.19838792085647583
Testing 9280/10449: Loss 0.1984175145626068
Testing 9290/10449: Loss 0.19850927591323853
Testing 9300/10449: Loss 0.19843526184558868
Testing 9310/10449: Loss 0.19846728444099426
Testing 9320/10449: Loss 0.19842220842838287
Testing 9330/10449: Loss 0.19834767282009125
Testing 9340/10449: Loss 0.19827218353748322
Testing 9350/10449: Loss 0.19828464090824127
Testing 9360/10449: Loss 0.1983703374862671
Testing 9370/10449: Loss 0.19853834807872772
Testing 9380/10449: Loss 0.19842667877674103
Testing 9390/10449: Loss 0.1985739916563034
Testing 9400/10449: Loss 0.19850026071071625
Testing 9410/10449: Loss 0.19858668744564056
Testing 9420/10449: Loss 0.19845998287200928
Testing 9430/10449: Loss 0.19843192398548126
Testing 9440/10449

KeyboardInterrupt: 

In [ ]:
split = 0
threshold = 0.2
with_softmax = False

validate(model_name, split, threshold, with_softmax)

# InceptionV4

In [8]:
model_name = 'inceptionv4'
split = 0
threshold = 0
with_softmax = False

validate(model_name, split, threshold, with_softmax)

Testing 0/10449: Loss 0.004624051041901112
Testing 10/10449: Loss 0.10310280323028564
Testing 20/10449: Loss 0.115280881524086
Testing 30/10449: Loss 0.10917960852384567
Testing 40/10449: Loss 0.10316640138626099
Testing 50/10449: Loss 0.10628500580787659
Testing 60/10449: Loss 0.12506847083568573
Testing 70/10449: Loss 0.1376490294933319
Testing 80/10449: Loss 0.1357140839099884
Testing 90/10449: Loss 0.13027217984199524
Testing 100/10449: Loss 0.14952149987220764
Testing 110/10449: Loss 0.14832019805908203
Testing 120/10449: Loss 0.16411101818084717
Testing 130/10449: Loss 0.1694299727678299
Testing 140/10449: Loss 0.16547007858753204
Testing 150/10449: Loss 0.17975389957427979
Testing 160/10449: Loss 0.1896299421787262
Testing 170/10449: Loss 0.1932884156703949
Testing 180/10449: Loss 0.189620241522789
Testing 190/10449: Loss 0.20746460556983948
Testing 200/10449: Loss 0.2063157558441162
Testing 210/10449: Loss 0.20778951048851013
Testing 220/10449: Loss 0.20368602871894836
Testing 

Testing 1860/10449: Loss 0.18734242022037506
Testing 1870/10449: Loss 0.18734030425548553
Testing 1880/10449: Loss 0.187532439827919
Testing 1890/10449: Loss 0.18786370754241943
Testing 1900/10449: Loss 0.18752993643283844
Testing 1910/10449: Loss 0.18869557976722717
Testing 1920/10449: Loss 0.18851666152477264
Testing 1930/10449: Loss 0.18809424340724945
Testing 1940/10449: Loss 0.18778298795223236
Testing 1950/10449: Loss 0.1876758337020874
Testing 1960/10449: Loss 0.19023050367832184
Testing 1970/10449: Loss 0.18989792466163635
Testing 1980/10449: Loss 0.19044576585292816
Testing 1990/10449: Loss 0.1906612366437912
Testing 2000/10449: Loss 0.19056390225887299
Testing 2010/10449: Loss 0.19043266773223877
Testing 2020/10449: Loss 0.19086620211601257
Testing 2030/10449: Loss 0.19035623967647552
Testing 2040/10449: Loss 0.18987204134464264
Testing 2050/10449: Loss 0.1897287368774414
Testing 2060/10449: Loss 0.18984663486480713
Testing 2070/10449: Loss 0.18947821855545044
Testing 2080/10

Testing 3700/10449: Loss 0.191061869263649
Testing 3710/10449: Loss 0.1910519003868103
Testing 3720/10449: Loss 0.19086988270282745
Testing 3730/10449: Loss 0.1910146176815033
Testing 3740/10449: Loss 0.1911129206418991
Testing 3750/10449: Loss 0.1908179223537445
Testing 3760/10449: Loss 0.19071482121944427
Testing 3770/10449: Loss 0.19078190624713898
Testing 3780/10449: Loss 0.19068357348442078
Testing 3790/10449: Loss 0.1906711310148239
Testing 3800/10449: Loss 0.1906542181968689
Testing 3810/10449: Loss 0.19071227312088013
Testing 3820/10449: Loss 0.1907120943069458
Testing 3830/10449: Loss 0.1905698925256729
Testing 3840/10449: Loss 0.19136719405651093
Testing 3850/10449: Loss 0.19157035648822784
Testing 3860/10449: Loss 0.19124090671539307
Testing 3870/10449: Loss 0.19128720462322235
Testing 3880/10449: Loss 0.19113107025623322
Testing 3890/10449: Loss 0.19098731875419617
Testing 3900/10449: Loss 0.19089829921722412
Testing 3910/10449: Loss 0.1907566785812378
Testing 3920/10449: L

Testing 5540/10449: Loss 0.19157814979553223
Testing 5550/10449: Loss 0.19142848253250122
Testing 5560/10449: Loss 0.1913250982761383
Testing 5570/10449: Loss 0.19152981042861938
Testing 5580/10449: Loss 0.19228917360305786
Testing 5590/10449: Loss 0.1922118365764618
Testing 5600/10449: Loss 0.19217891991138458
Testing 5610/10449: Loss 0.1921454221010208
Testing 5620/10449: Loss 0.19214500486850739
Testing 5630/10449: Loss 0.19242262840270996
Testing 5640/10449: Loss 0.19243019819259644
Testing 5650/10449: Loss 0.19250667095184326
Testing 5660/10449: Loss 0.19255094230175018
Testing 5670/10449: Loss 0.1925053745508194
Testing 5680/10449: Loss 0.19239404797554016
Testing 5690/10449: Loss 0.19245545566082
Testing 5700/10449: Loss 0.19275254011154175
Testing 5710/10449: Loss 0.1926986575126648
Testing 5720/10449: Loss 0.1926165223121643
Testing 5730/10449: Loss 0.19263365864753723
Testing 5740/10449: Loss 0.19246000051498413
Testing 5750/10449: Loss 0.19235338270664215
Testing 5760/10449:

KeyboardInterrupt: 

In [ ]:
split = 0
threshold = 0.1
with_softmax = False

validate(model_name, split, threshold, with_softmax)

In [9]:
split = 0
threshold = 0.15
with_softmax = False

validate(model_name, split, threshold, with_softmax)

Testing 0/10449: Loss 0.008685765787959099
Testing 10/10449: Loss 0.09678062051534653
Testing 20/10449: Loss 0.10734247416257858
Testing 30/10449: Loss 0.12644322216510773
Testing 40/10449: Loss 0.10777633637189865
Testing 50/10449: Loss 0.11307443678379059
Testing 60/10449: Loss 0.12657465040683746
Testing 70/10449: Loss 0.1268310695886612
Testing 80/10449: Loss 0.13000907003879547
Testing 90/10449: Loss 0.12832027673721313
Testing 100/10449: Loss 0.14678515493869781
Testing 110/10449: Loss 0.14491300284862518
Testing 120/10449: Loss 0.14514806866645813
Testing 130/10449: Loss 0.14641451835632324
Testing 140/10449: Loss 0.14501696825027466
Testing 150/10449: Loss 0.15248891711235046
Testing 160/10449: Loss 0.14993184804916382
Testing 170/10449: Loss 0.15237122774124146
Testing 180/10449: Loss 0.14605611562728882
Testing 190/10449: Loss 0.16019105911254883
Testing 200/10449: Loss 0.16820292174816132
Testing 210/10449: Loss 0.1715654730796814
Testing 220/10449: Loss 0.17227311432361603


KeyboardInterrupt: 

In [10]:
split = 0
threshold = 0.2
with_softmax = False

validate(model_name, split, threshold, with_softmax)

Testing 0/10449: Loss 0.011830921284854412
Testing 10/10449: Loss 0.09283148497343063
Testing 20/10449: Loss 0.1037432923913002
Testing 30/10449: Loss 0.08554648607969284
Testing 40/10449: Loss 0.07829305529594421
Testing 50/10449: Loss 0.09093821793794632
Testing 60/10449: Loss 0.10998076945543289
Testing 70/10449: Loss 0.11758489906787872
Testing 80/10449: Loss 0.11943758279085159
Testing 90/10449: Loss 0.11941198259592056
Testing 100/10449: Loss 0.1323235034942627
Testing 110/10449: Loss 0.13507606089115143
Testing 120/10449: Loss 0.14162302017211914
Testing 130/10449: Loss 0.1496129035949707
Testing 140/10449: Loss 0.14873261749744415
Testing 150/10449: Loss 0.16189441084861755
Testing 160/10449: Loss 0.16603146493434906
Testing 170/10449: Loss 0.16652807593345642
Testing 180/10449: Loss 0.16250762343406677
Testing 190/10449: Loss 0.17159022390842438
Testing 200/10449: Loss 0.17281877994537354
Testing 210/10449: Loss 0.17567329108715057
Testing 220/10449: Loss 0.17377310991287231
T

Testing 1860/10449: Loss 0.17970474064350128
Testing 1870/10449: Loss 0.17995722591876984
Testing 1880/10449: Loss 0.18025894463062286
Testing 1890/10449: Loss 0.1801525056362152
Testing 1900/10449: Loss 0.1806333214044571
Testing 1910/10449: Loss 0.18129894137382507
Testing 1920/10449: Loss 0.18180853128433228
Testing 1930/10449: Loss 0.18142417073249817
Testing 1940/10449: Loss 0.1811562478542328
Testing 1950/10449: Loss 0.1815924048423767
Testing 1960/10449: Loss 0.18385672569274902
Testing 1970/10449: Loss 0.18361572921276093
Testing 1980/10449: Loss 0.18394939601421356
Testing 1990/10449: Loss 0.1840784102678299
Testing 2000/10449: Loss 0.18405583500862122
Testing 2010/10449: Loss 0.184295192360878
Testing 2020/10449: Loss 0.18540754914283752
Testing 2030/10449: Loss 0.18501417338848114
Testing 2040/10449: Loss 0.18464244902133942
Testing 2050/10449: Loss 0.18485847115516663
Testing 2060/10449: Loss 0.1849316507577896
Testing 2070/10449: Loss 0.1848662942647934
Testing 2080/10449:

Testing 3700/10449: Loss 0.1859242022037506
Testing 3710/10449: Loss 0.18599194288253784
Testing 3720/10449: Loss 0.18584448099136353
Testing 3730/10449: Loss 0.18604817986488342
Testing 3740/10449: Loss 0.1859780102968216
Testing 3750/10449: Loss 0.18580776453018188
Testing 3760/10449: Loss 0.18572373688220978
Testing 3770/10449: Loss 0.1857353299856186
Testing 3780/10449: Loss 0.18561574816703796
Testing 3790/10449: Loss 0.1856975108385086
Testing 3800/10449: Loss 0.18567049503326416
Testing 3810/10449: Loss 0.18544696271419525
Testing 3820/10449: Loss 0.1854935884475708
Testing 3830/10449: Loss 0.1853601187467575
Testing 3840/10449: Loss 0.1857604682445526
Testing 3850/10449: Loss 0.1856146603822708
Testing 3860/10449: Loss 0.18531973659992218
Testing 3870/10449: Loss 0.1852555274963379
Testing 3880/10449: Loss 0.18518903851509094
Testing 3890/10449: Loss 0.18524935841560364
Testing 3900/10449: Loss 0.18542571365833282
Testing 3910/10449: Loss 0.18530520796775818
Testing 3920/10449:

Testing 5530/10449: Loss 0.19108302891254425
Testing 5540/10449: Loss 0.19122324883937836
Testing 5550/10449: Loss 0.19110217690467834
Testing 5560/10449: Loss 0.19094926118850708
Testing 5570/10449: Loss 0.19120827317237854
Testing 5580/10449: Loss 0.19126415252685547
Testing 5590/10449: Loss 0.19123098254203796
Testing 5600/10449: Loss 0.19115009903907776
Testing 5610/10449: Loss 0.191244974732399
Testing 5620/10449: Loss 0.19117316603660583
Testing 5630/10449: Loss 0.19103997945785522
Testing 5640/10449: Loss 0.19102440774440765
Testing 5650/10449: Loss 0.19121719896793365
Testing 5660/10449: Loss 0.19119881093502045
Testing 5670/10449: Loss 0.1916203796863556
Testing 5680/10449: Loss 0.19151391088962555
Testing 5690/10449: Loss 0.19150033593177795
Testing 5700/10449: Loss 0.1914549469947815
Testing 5710/10449: Loss 0.19135864078998566
Testing 5720/10449: Loss 0.1912645399570465
Testing 5730/10449: Loss 0.19109462201595306
Testing 5740/10449: Loss 0.19100147485733032
Testing 5750/10

Testing 7370/10449: Loss 0.1926269829273224
Testing 7380/10449: Loss 0.19292518496513367
Testing 7390/10449: Loss 0.1930798590183258
Testing 7400/10449: Loss 0.1932721734046936
Testing 7410/10449: Loss 0.19353650510311127
Testing 7420/10449: Loss 0.19338184595108032
Testing 7430/10449: Loss 0.19330617785453796
Testing 7440/10449: Loss 0.19324271380901337
Testing 7450/10449: Loss 0.19340264797210693
Testing 7460/10449: Loss 0.19367864727973938
Testing 7470/10449: Loss 0.19362160563468933
Testing 7480/10449: Loss 0.19346027076244354
Testing 7490/10449: Loss 0.19354593753814697
Testing 7500/10449: Loss 0.1937277913093567
Testing 7510/10449: Loss 0.19387781620025635
Testing 7520/10449: Loss 0.1937739998102188
Testing 7530/10449: Loss 0.19361308217048645
Testing 7540/10449: Loss 0.1935032159090042
Testing 7550/10449: Loss 0.19348813593387604
Testing 7560/10449: Loss 0.1934482604265213
Testing 7570/10449: Loss 0.1937921792268753
Testing 7580/10449: Loss 0.19373555481433868
Testing 7590/10449

Testing 9210/10449: Loss 0.195419043302536
Testing 9220/10449: Loss 0.19537131488323212
Testing 9230/10449: Loss 0.19538258016109467
Testing 9240/10449: Loss 0.1954733431339264
Testing 9250/10449: Loss 0.19543901085853577
Testing 9260/10449: Loss 0.19541499018669128
Testing 9270/10449: Loss 0.1953100711107254
Testing 9280/10449: Loss 0.1952821910381317
Testing 9290/10449: Loss 0.19532959163188934
Testing 9300/10449: Loss 0.19528190791606903
Testing 9310/10449: Loss 0.1953705996274948
Testing 9320/10449: Loss 0.19547779858112335
Testing 9330/10449: Loss 0.19534291326999664
Testing 9340/10449: Loss 0.19536307454109192
Testing 9350/10449: Loss 0.19528743624687195
Testing 9360/10449: Loss 0.19573554396629333
Testing 9370/10449: Loss 0.19620594382286072
Testing 9380/10449: Loss 0.19607198238372803
Testing 9390/10449: Loss 0.1961653232574463
Testing 9400/10449: Loss 0.19607025384902954
Testing 9410/10449: Loss 0.19596196711063385
Testing 9420/10449: Loss 0.1958128660917282
Testing 9430/10449

(tensor(0.1955, device='cuda:0'), 0.584009765361956)

# Polynet

In [5]:
model_name = 'polynet'
split = 0
threshold = 0
with_softmax = False

validate(model_name, split, threshold, with_softmax)

Testing 0/10449: Loss 0.0765305906534195
Testing 10/10449: Loss 0.0715467631816864
Testing 20/10449: Loss 0.06818187236785889
Testing 30/10449: Loss 0.07135160267353058
Testing 40/10449: Loss 0.07376623153686523
Testing 50/10449: Loss 0.07723189890384674
Testing 60/10449: Loss 0.08459264785051346
Testing 70/10449: Loss 0.1077052652835846
Testing 80/10449: Loss 0.12952475249767303
Testing 90/10449: Loss 0.1280219405889511
Testing 100/10449: Loss 0.13671785593032837
Testing 110/10449: Loss 0.13829031586647034
Testing 120/10449: Loss 0.14067405462265015
Testing 130/10449: Loss 0.14521706104278564


KeyboardInterrupt: 

In [ ]:
model_name = 'polynet'
split = 0
threshold = 0.1
with_softmax = False

validate(model_name, split, threshold, with_softmax)

In [ ]:
model_name = 'polynet'
split = 0
threshold = 0.15
with_softmax = False

validate(model_name, split, threshold, with_softmax)

In [ ]:
model_name = 'polynet'
split = 0
threshold = 0.2
with_softmax = False

validate(model_name, split, threshold, with_softmax)

In [5]:
import pretrainedmodels

model = pretrainedmodels.__dict__['dpn68'](pretrained = 'imagenet')

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/dpn68-66bebafa7.pth" to /home/saa2/.torch/models/dpn68-66bebafa7.pth
100%|██████████| 50761994/50761994 [00:03<00:00, 12702345.14it/s]


In [ ]:
mo

VGG(
  (linear0): Linear(in_features=25088, out_features=4096, bias=True)
  (relu0): ReLU(inplace)
  (dropout0): Dropout(p=0.5)
  (linear1): Linear(in_features=4096, out_features=4096, bias=True)
  (relu1): ReLU(inplace)
  (dropout1): Dropout(p=0.5)
  (last_linear): Linear(in_features=4096, out_features=28, bias=True)
)

In [5]:
in_features = model.last_linear.in_features
model.last_linear = torch.nn.Linear(in_features, 28)

In [11]:
model.load_state_dict(torch.load('vgg19_bn_0_40.pth.tar')['state_dict'])

In [10]:
torch.

OrderedDict([('_features.0.weight',
              tensor([[[[-3.2130e-08, -3.1717e-08, -6.7120e-08],
                        [-2.9321e-08, -2.1596e-08, -5.3207e-08],
                        [-5.3304e-08, -4.0832e-08, -6.6434e-08]],
              
                       [[ 1.3710e-07,  1.2848e-07,  7.1270e-08],
                        [ 1.5487e-07,  1.5186e-07,  8.3582e-08],
                        [ 9.3392e-08,  9.9451e-08,  4.1284e-08]],
              
                       [[ 1.7435e-07,  1.7840e-07,  1.0561e-07],
                        [ 1.9644e-07,  1.9769e-07,  1.2560e-07],
                        [ 1.1935e-07,  1.3447e-07,  5.9638e-08]]],
              
              
                      [[[-6.3467e-08, -5.2714e-08, -3.8511e-08],
                        [-1.4349e-08, -9.0426e-09, -1.7164e-08],
                        [ 2.2282e-08,  1.9084e-08,  1.8150e-09]],
              
                       [[-1.2191e-08, -6.1933e-09,  8.5022e-09],
                        [ 3.8618e-08,  

In [8]:
model.train()

VGG(
  (_features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3